## BIOM262: ChIP-Seq workshop – Part 2

---
Start by grabbing an interactive node: 
```
qsub -I -l nodes=1:ppn=8 -l walltime=4:00:00 -q hotel![image.png](attachment:image.png)
```

---

Following the steps 1-5 on we did on Tuesday, you should have a tagdirectory for each of the datasets (6 in total) and you should also have the UCSCfiles.   

As we discussed, each dataset was created by a different antibody, and they can be divided into three types: TFs (transcription factors) HMs (histone modifications) and global input (background). Since we will need to treat each type differently, I recommend making a directory for each – input, TFs and HMs and move the tag directories to the relevant one (e.g. tfs/oct4-esc/, etc.).

---
**6. One of the most common tasks with ChIP-seq data is to find ‘enriched’ regions commonly called “peaks”.** HOMER contains a command called findPeaks which is used to analyze tag directories for peaks. There are two common ways to use the command:

```
findPeaks <tag directory> -i <control tag directory>  -style factor  -o auto
```

or

```
findPeaks <path>/oct4-esc/ -i <path>/input-esc/ -style factor -o auto
```

The difference between the two is in the “-style factor/histone” argument, which will tell the program to look for focal, fixed width peaks vs. variable length peaks; the later is more common in the case of histone modifications. To find Oct4 peaks in the data, run the following command:

```
findPeaks <path>/oct4-esc/ -i <path>/input-esc/ -style factor -o auto
```
  
This command will look for enriched regions and filter them based on several criterion, including ensuring that they have at least 4-fold more reads in peak regions relative to the control experiment (in this case `input-esc/`). The output will be stored in a HOMER-style peak file located in the Oct4 tag directory (`oct4-esc/peaks.txt`). The beginning of this file contains statistics and QC stats from the peak finding, including the number of peaks, number of peaks lost to input filtering, etc.     

One field worth paying attention to is the **“Approximate IP efficiency”** which reports what fraction of reads from the experiment were actually found in peaks. For most decent experiments this value ranges from 1% to >30% (remember ChIP is an enrichment strategy... there is plenty of background in the data too!). Below this are the peaks along with enrichment statistics for each region.

One other thing to note is that HOMER reports the results in a ‘peak’ file, which has a slightly different format from a traditional BED file format. To create a BED file from the peak file, use the tool pos2bed.pl (e.g. `pos2bed.pl oct4-esc/peaks.txt > oct4-esc.bed`. The “`> output.txt`” part at the end means that the results will be sent to stdout, and the “`> output.txt`” is used to capture the output information in a file.). BED files can be uploaded to IGV just like a bedGraph file. Also, most HOMER programs will work with either BED or peak files as input.


Next we will find peaks for all samples using two ‘for loops’ – for the two types of data:

```
for dir in <path>/hms/*; 
do findPeaks $dir -i <path>input-esc/ -style histone -o auto; 
done 
```

and

```
for dir in <path>/tfs/*; 
do findPeaks $dir -i <path>input-esc/ -style factor -o auto; 
done
```

Make a directory for the annotation files (e.g., “annotations”) and convert the peak.txt and region.txt files to bed files:

```
for dir in <path>/hms/*; 
do dirname=${dir##*/}; 
    pos2bed.pl $dir/regions.txt > <path>/annotations/$dirname.bed; 
done
```

And 

```
for dir in <path>/tfs/*; 
do dirname=${dir##*/}; 
    pos2bed.pl $dir/peaks.txt > <path>/annotations/$dirname.bed; 
done
```

Copy to your local computer using scp or filezila and load to IGV. Explore the original (previous to conversion to bed) peaks.txt and regions.txt files with less -S in the command line. For instance, each peak gets a score, see how the ones with high scores look on IGV vs. ones with low scores. 

---
**7. Now that we have identified peaks from our ChIP-seq data, it is time to figure out more information about where they are and what genes they might be regulating.** 

HOMER contains a program called annotatePeaks.pl that performs a wide variety of functions using peak/BED files. First, lets use it to perform basic annotation of the peak file. The annotatePeaks.pl program works like this:


```
annotatePeaks.pl <peak/BED file> <genome version> [options] > output.txt
```

To annotate peaks from the Oct4 experiment:

```
annotatePeaks.pl <path>/oct4-esc/peaks.txt mm9 > <path>/annotations/oct4.annotation.txt
```

If we view the `“oct4.annotation.txt”` file with `less -S`, you’ll see several annotation columns. Take note of the columns specifying the nearest gene TSS, the distance, and the annotation of the genomic region the peak is located in. This annotation is split into two separate columns – one is basic (i.e., exon, promoter, intergenic, intron etc.), and a more detailed annotation that describes CpG islands, repeat elements, etc. You might have also noticed while the command was running that stats about annotation enrichment too.

You can do it using a for loop for the two types of datasets:

```
for dir in <path>/hms/*; 
do dirname=${dir##*/}; 
    annotatePeaks.pl $dir/regions.txt mm9  > <path>/annotations/$dirname.annotation.txt; 
done

```

and for tfs:


```
for dir in <path>/tfs/*; 
do dirname=${dir##*/}; 
    annotatePeaks.pl $dir/peaks.txt mm9  > <path>/annotations/$dirname.annotation.txt; 
done
```

---
**8. The annotatePeaks.pl program can also be used to create histograms that display the read enrichment relative to given genomic features**, including transcription start sites (TSS) or any other set of regions the user wants to define. Since the TSS is so commonly used for this purpose, HOMER has a built-in annotation for TSS (based on RefSeq transcripts). The key parameters to create a histogram are the `-hist #` and `-size #` options, which control the binning size and total length of the histogram. The other important option is the `-d <tag directory>`, which specifies which experiments to compile histograms for. In general:

```
annotatePeaks.pl <peak/BED file> <genome version> -size <#> -hist <#> -d <Tag Directory> > output.txt
```
  
(note that the peak/BED file can be replaced with the key word “tss” to make a histogram at the TSS). To create a histogram near the TSS with the data from the experiments we’ve looked at thus far, run the following:

```
annotatePeaks.pl tss mm9 -size 8000 -hist 10 -d <path>/tagdir/*/* > <path>/annotations/tss-histogram.txt
```

Open the “tss-histogram.txt” using a spreadsheet program (R, Excel or similar). You’ll notice that the first column gives the distance offsets from the TSS followed by columns corresponding to the ‘coverage’, ‘+ Tags’, and ‘- Tags’ for each experiment. Try graphing each as X-Y line graph using the first column as the X-coordinate to see the patterns.



---
### Motif finding

**9. DNA motif finding is a powerful technique to analyze ChIP-seq experiments.** Unlike gene expression data, ChIP-seq localizes signals to very specific regions of the genome allowing for accurate identification of the genetic signals responsible for recruiting various transcription factors. To use HOMER’s motif analysis program, run the findMotifsGenome.pl command using peak files from the experiments. In general the command works like this:

```
findMotifsGenome.pl <peak/BED file> <genome version> <output directory> [options]
```

Common options for motif finding are `-p <#cpu>` for parallel execution, and `-size <#>` to specify the size of the regions you which to search for DNA motifs. For transcription factor motifs, a good size is 100. Make a new directory (e.g., “motifs”). To find enriched motifs, run the following for loop:

```
for dir in <path>/tfs/*; 
do dirname=${dir##*/}; 
    mkdir <path>/motifs/$dirname; findMotifsGenome.pl $dir/peaks.txt mm9  <path>/motifs/$dirname/ -mask -size 100 -p 10; 
done
```

This command will perform several different steps, including checking for the enrichment of a library of known transcription factor motifs as well as perform a de novo search for enriched motifs. The “-mask” tells the program to mask repeat sequences in the genome. Depending on the speed of your server this program may take while to run. Once it’s finished, copy the homerResults.html files to your computer, and use your Internet Browser to open it.

To copy the entire directories:

```
scp -r <user>@tscc.sdsc.edu:<path>/motifs/* <directory on your local machine>
```

This file will list the top de novo motifs found as well as provide stats and best matches to known transcription factor motifs. A second file called knownResults.html contains the enrichment statistics for known motifs. 

You might notice that both analyses indicate that Oct4 and Sox2 peaks in ESC are highly enriched for an OCT:SOX composite motif, which has been shown to be very important in establishing pluripotent enhancers.